In [1]:
#Dependencies
import glob
import numpy as np
import pandas as pd
import seaborn as sb

from pylab import rcParams

In [16]:
#Create "hudl" class to scrape football data 
#from hudl
class Hudl:
    #Initializer/Instance
    def __init__(self):
        import pandas as pd
        game = input("Which game would you like to break down?")
        pd.read_excel(game + ".xlsx")
#         print(game + ".csv")
        pass
        
    #Down/Distance Crosstab
    def downDist(self, downs, dist, form, play):
        self.downs 
        pass

In [58]:
hudl = Hudl()
hudl

Which game would you like to break down?


FileNotFoundError: [Errno 2] No such file or directory: '.xlsx'

In [2]:
#Set dimensions for plotting
%matplotlib inline
rcParams["figure.figsize"] = 5,4
sb.set_style("whitegrid")

In [3]:
def breakdown(*games):
    game = pd.read_excel(*games)
#     print(game)
    print(type(game))
    return game
breakdown("WPU_RPI_2017.xlsx")

<class 'pandas.core.frame.DataFrame'>


,PLAY #,ODK,DN,DIST,HASH,YARD LN,PLAY TYPE,RESULT,GN/LS,OFF FORM,OFF PLAY,OFF STR,PLAY DIR,GAP,PASS ZONE,DEF FRONT,COVERAGE,BLITZ,QTR
0,1,K,NaN,NaN,NaN,NaN,KO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,D,1.0,10.0,R,-14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UN,4.0,NaN,NaN
3,4,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,D,2.0,11.0,R,-13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OV,4.0,NaN,NaN
5,6,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,D,3.0,14.0,R,-10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UN,4.0,NaN,NaN
7,8,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,K,4.0,12.0,R,-12.0,Punt Rec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,O,1.0,10.0,M,37.0,Run,NaN,8.0,QUADS BONE,JET SWEEP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
def downDist(*games):
    for game in games:
        game = pd.read_excel(*games)
        DD = pd.crosstab([game["DN"], 
                     game["DIST"]], 
                    [game["OFF FORM"], 
                     game["OFF PLAY"]])
        DD.dropna()
        DD
        
        return DD
downDist("WPU_RPI_2017.xlsx")

OFF FORM 3X2          3X2 BONE 3X2 WING      JOKER                     \
OFF PLAY 3ST QB DRAW JET SWEEP      3ST 5ST CTR SW IZ RD JET SWEEP OZ   
DN  DIST                                                                
1.0 5.0    0       0         0        0   0      0     0         1  0   
    6.0    0       0         0        0   0      0     0         0  0   
    9.0    0       0         0        0   0      0     0         0  0   
    10.0   0       0         0        0   0      1     1         0  1   
    15.0   0       0         0        0   0      0     0         0  0   
    16.0   0       0         0        0   0      0     0         0  0   
2.0 5.0    0       0         0        0   0      0     0         0  0   
    6.0    0       0         0        0   0      0     0         0  0   
    9.0    0       0         1        0   0      0     0         0  0   
    10.0   0       1         0        0   0      0     0         0  0   
    15.0   0       0         0        0   1      0     0         0  0   
    1.0    0       0         0        0   0      0     0         0  0   
    2.0    1       0         0        0   0      0     0         0  0   
    3.0    0       0         1        1   0      0     0         0  0   
    4.0    0       0         0        0   0      0     0         0  0   
    7.0    0       0         0        0   1      0     0         0  0   
    8.0    0       0         0        0   0      0     0         0  0   
    14.0   0       0         0        1   0      0     0         0  0   
3.0 5.0    0       0         0        0   0      0     0         0  0   
    6.0    0       0         0        0   0      0     0         0  0   
    1.0    0       0         0        1   0      0     0         0  0   
    2.0    0       0         0        0   0      0     0         0  0   
    4.0    0       0         0        0   0      0     0         0  0   
    7.0    0       0         0        0   0      0     0         0  0   
    14.0   0       0         0        0   0      0     0         0  0   
4.0 1.0    0       0         0        0   0      0     0         0  0   
    8.0    0       0         0        0   0      0     0         0  0   

OFF FORM KINGS OPEN   ...   TWINS OPEN HARDER TWINS OPEN SOFTER               \
OFF PLAY         IZ   ...               IZ RD               3ST PA JET SWEEP   
DN  DIST              ...                                                      
1.0 5.0           0   ...                   0                 0            0   
    6.0           0   ...                   0                 0            0   
    9.0           1   ...                   0                 0            0   
    10.0          2   ...                   0                 1            1   
    15.0          0   ...                   0                 0            0   
    16.0          0   ...                   0                 0            0   
2.0 5.0           0   ...                   0                 0            0   
    6.0           0   ...                   1                 0            0   
    9.0           0   ...                   0                 0            0   
    10.0          0   ...                   0                 0            1   
    15.0          0   ...                   0                 0            0   
    1.0           0   ...                   0                 0            0   
    2.0           0   ...                   0                 0            0   
    3.0           0   ...                   0                 0            0   
    4.0           0   ...                   0                 0            0   
    7.0           0   ...                   0                 0            0   
    8.0           0   ...                   0                 0            0   
    14.0          0   ...                   0                 0            0   
3.0 5.0           0   ...                   0                 0            0   
    6.0           0   ...                   0    

In [21]:
def playNum(*games):
    #Combine all xlsx files into 1 Dataframe
    all_data = pd.DataFrame()
    for f in glob.glob(r"C:\Users\User\Desktop\Personal_Projects\Gameplay_Analysis\*.xlsx"):
        df = pd.read_excel(f)
        all_data = all_data.append(df, ignore_index=True)
        df = all_data[["PLAY #","YARD LN","DN", "DIST", 
                   "OFF FORM", "OFF PLAY", "PLAY TYPE", "GN/LS"]]
        table = df.dropna(subset=["OFF PLAY"])
        table["DD"] = table[["DN", "DIST"]].astype(str).sum(axis=1)
        table["playNum"] = table["PLAY #"]
        table["DOWN"] = table["DN"]
        table["ydLine"] = table["YARD LN"]
        table = table[["PLAY #","DN","playNum","ydLine","DOWN",
                       "DIST","DD","GN/LS","PLAY TYPE", "OFF FORM", 
                       "OFF PLAY"]].set_index(["PLAY #", "DN"])
        return table
playNum()

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

,,playNum,ydLine,DOWN,DIST,DD,GN/LS,PLAY TYPE,OFF FORM,OFF PLAY
PLAY #,DN,,,,,,,,,
2,1.0,2,-29.0,1.0,10.0,1.010.0,4.0,Run,TWINS OPEN HARD,POWER
3,2.0,3,-33.0,2.0,7.0,2.07.0,0.0,Pass,TWINS OPEN SOFT,PA POWER
4,3.0,4,-33.0,3.0,7.0,3.07.0,-10.0,Pass,QUEENS OPEN,RB SCREEN WK
5,3.0,5,-23.0,3.0,17.0,3.017.0,NaN,Run,QUEENS OPEN,QB POWER
7,1.0,7,38.0,1.0,10.0,1.010.0,0.0,Run,SPREAD BONE T OVER,POWER
8,2.0,8,38.0,2.0,10.0,2.010.0,2.0,Run,PRO WING,BUCK SWEEP
9,3.0,9,36.0,3.0,8.0,3.08.0,5.0,Pass,QUADS,3ST
10,4.0,10,31.0,4.0,2.0,4.02.0,NaN,Run,QUEENS OPEN,QB POWER
11,1.0,11,25.0,1.0,10.0,1.010.0,1.0,Run,QUEENS OPEN,INF TRAP


In [20]:
def effective(*games):
    #Combine all xlsx files into 1 Dataframe
    all_data = pd.DataFrame()
    for f in glob.glob(r"C:\Users\User\Desktop\Personal_Projects\Gameplay_Analysis\*.xlsx"):
        df = pd.read_excel(f)
        all_data = all_data.append(df, ignore_index=True)
    all_games = all_data[["PLAY #","YARD LN","DN", "DIST", "OFF FORM", 
               "OFF PLAY", "PLAY TYPE", "GN/LS"]]
    table = all_games.dropna(subset=["OFF FORM"])
    table["DD"] = table[["DN", "DIST"]].astype(str).sum(axis=1)
    table["playNum"] = table["PLAY #"]
    table["DOWN"] = table["DN"]
    table["ydLine"] = table["YARD LN"]
    table = table[["PLAY #","DN","playNum","ydLine","DOWN",
                   "DIST","DD","GN/LS","PLAY TYPE", "OFF FORM", 
                   "OFF PLAY"]].set_index(["PLAY #", "DN"])
    table.fillna(value = table["GN/LS"].median(), inplace=True)
    conditions = [
    ((table["GN/LS"] >=4) & (table["DD"] == "1.010.0")), 
    (table["GN/LS"] >= (table["DIST"]/2)), 
    (table["GN/LS"] < (table["DIST"]/2))
    ]
    choices = ["Yes", "Yes", "No"]
    table["EFFECTIVE"] = np.select(conditions, choices)
    rpiOff = table.to_csv("rpiOFF.csv")
    return table
effective()

C:\Users\User\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas

,,playNum,ydLine,DOWN,DIST,DD,GN/LS,PLAY TYPE,OFF FORM,OFF PLAY,EFFECTIVE
PLAY #,DN,,,,,,,,,,
2.0,1.0,2.0,-29.0,1.0,10.0,1.010.0,4.0,Run,TWINS OPEN HARD,POWER,Yes
3.0,2.0,3.0,-33.0,2.0,7.0,2.07.0,0.0,Pass,TWINS OPEN SOFT,PA POWER,No
4.0,3.0,4.0,-33.0,3.0,7.0,3.07.0,-10.0,Pass,QUEENS OPEN,RB SCREEN WK,No
5.0,3.0,5.0,-23.0,3.0,17.0,3.017.0,3.0,Run,QUEENS OPEN,QB POWER,No
7.0,1.0,7.0,38.0,1.0,10.0,1.010.0,0.0,Run,SPREAD BONE T OVER,POWER,No
8.0,2.0,8.0,38.0,2.0,10.0,2.010.0,2.0,Run,PRO WING,BUCK SWEEP,No
9.0,3.0,9.0,36.0,3.0,8.0,3.08.0,5.0,Pass,QUADS,3ST,Yes
10.0,4.0,10.0,31.0,4.0,2.0,4.02.0,3.0,Run,QUEENS OPEN,QB POWER,Yes
11.0,1.0,11.0,25.0,1.0,10.0,1.010.0,1.0,Run,QUEENS OPEN,INF TRAP,No


In [ ]:
import glob

def allgames(): 
    #Combine all xlsx files into 1 Dataframe
    all_data = pd.DataFrame()
    for f in glob.glob(r"C:\Users\Michael Diaz\Desktop\Personal_Projects\Analysis\*.xlsx"):
        df = pd.read_excel(f)
        all_data = all_data.append(df, ignore_index=True)
        return all_data
allgames()

In [ ]:
#Individual Games
glob.glob(r"C:/Users/Michael Diaz/Desktop/Personal_Projects/Analysis/*.xlsx")

In [ ]:
#Combine all xlsx files into 1 Dataframe
all_data = pd.DataFrame()
for f in glob.glob(r"C:\Users\Michael Diaz\Desktop\Personal_Projects\Analysis\*.xlsx"):
    df = pd.read_excel(f)
    all_data = all_data.append(df, ignore_index=True)
all_data.describe()
all_data.head()

In [9]:
def playByPlay():
    #Combine all xlsx files into 1 Dataframe
    all_data = pd.DataFrame()
    for f in glob.glob(r"C:\Users\User\Desktop\Personal_Projects\Gameplay_Analysis\*.xlsx"):
        df = pd.read_excel(f)
        all_data = all_data.append(df, ignore_index=True)
    #Predict current and following playcall
    all_data = all_data.dropna(subset=["OFF FORM"])
    all_data["PREVIOUS PLAY"] = all_data["OFF PLAY"].shift(1)
    all_data["prevPlay"] = all_data["PREVIOUS PLAY"]
    all_data["currentPlay"] = all_data["OFF PLAY"]
    prediction = all_data[["prevPlay", "currentPlay"]]
#     rpiOff = table.to_csv("rpiOFF.csv")
    return prediction
playByPlay()

C:\Users\User\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


,prevPlay,currentPlay
1,NaN,POWER
2,POWER,PA POWER
3,PA POWER,RB SCREEN WK
4,RB SCREEN WK,QB POWER
5,QB POWER,POWER
6,POWER,BUCK SWEEP
7,BUCK SWEEP,3ST
8,3ST,QB POWER
9,QB POWER,INF TRAP
11,INF TRAP,PA POWER


In [ ]:
def prediction():
    #Combine all xlsx files into 1 Dataframe
    all_data = pd.DataFrame()
    for f in glob.glob(r"C:\Users\Michael Diaz\Desktop\Personal_Projects\Analysis\*.xlsx"):
        df = pd.read_excel(f)
        all_data = all_data.append(df, ignore_index=True)
    #Predict current and following playcall
    all_data = all_data.dropna(subset=["OFF FORM"])
    all_data["PREVIOUS PLAY"] = all_data["OFF PLAY"].shift(1)
    all_data["prevPlay"] = all_data["PREVIOUS PLAY"]
    all_data["nextPlay"] = all_data["OFF PLAY"]
#     prediction = all_data[["prevPlay", "currentPlay"]]
    prediction = pd.crosstab(all_data["prevPlay"], all_data["nextPlay"])
#     rpiOff = table.to_csv("rpiOFF.csv")
    return prediction
prediction()

In [22]:
def scenarios(*games):
    #Combine all xlsx files into 1 Dataframe
    all_data = pd.DataFrame()
    for f in glob.glob(r"C:\Users\User\Desktop\Personal_Projects\Gameplay_Analysis\*.xlsx"):
        df = pd.read_excel(f)
        all_data = all_data.append(df, ignore_index=True)
    all_games = all_data[["PLAY #","YARD LN","DN", "DIST", "OFF FORM", 
               "OFF PLAY", "PLAY TYPE", "GN/LS"]]
    table = all_games.dropna(subset=["OFF FORM"])
    table["DD"] = table[["DN", "DIST"]].astype(str).sum(axis=1)
    table["playNum"] = table["PLAY #"]
    table["DOWN"] = table["DN"]
    table["ydLine"] = table["YARD LN"]
    table = table[["PLAY #","DN","playNum","ydLine","DOWN",
                   "DIST","DD","GN/LS","PLAY TYPE", "OFF FORM", 
                   "OFF PLAY"]].set_index(["PLAY #", "DN"])
    table.fillna(value = table["GN/LS"].median(), inplace=True)
    
    #Down/Distance Scenarios
#     all_data = all_data.dropna(subset=["OFF FORM"])
    table["PREVIOUS PLAY"] = table["OFF PLAY"].shift(1)
    table["prevPlay"] = table["PREVIOUS PLAY"]
    table["currentPlay"] = table["OFF PLAY"]
    ddConditions = [
    ((table["GN/LS"] > 0) & (table["DOWN"] == 1.0) & (table["DOWN"].shift(1) == 3.0)),
    ((table["GN/LS"] > 0) & (table["DOWN"] == 1.0) & (table["DOWN"].shift(1) == 2.0)),
    ((table["GN/LS"] > 0) & (table["DOWN"] == 1.0) & (table["DOWN"].shift(1) == 4.0) & (table["DIST"].shift(1) <= 3.0)), 
    ((table["GN/LS"] > 0) & (table["DOWN"] == 1.0) & (table["DOWN"].shift(1) == 4.0) & (table["DIST"].shift(1) >= 4.0))
    ]
    ddChoices = ["3rd Down Conversion", "2nd Down Conversion", 
               "4th and Short Conversion", "4th and Long Conversion"]
    table["Down/Dist Scenarios"] = np.select(ddConditions, ddChoices)
    
    #Game Changing Scenarios
    gcConditions = [
    (table["GN/LS"] > 30)
    ]
    gcChoices = ["Run/Pass > 30 yds"]
    table["Game Changing Scenarios"] = np.select(gcConditions, gcChoices)
    
    #Run/Pass Scenarios
    rpConditions = [
    ((table["GN/LS"] > 0) & (table["DD"] == "1.010.0")), 
    (table["GN/LS"] < 0), 
    (table["GN/LS"] < 5), 
    (table["GN/LS"] > 10), 
    ((table["GN/LS"] > 0) & (table["PLAY TYPE"] == "Pass")),
    ((table["GN/LS"] > 0) & (table["PLAY TYPE"] == "NaN")),
    ((table["GN/LS"] > 10) & (table["PLAY TYPE"] == "Pass"))
    ]
    rpChoices = ["Run for 1st Down", "Run < 0 yds", "Run < 5 yds", "Run < 10 yds", 
                  "Pass Complete", "Pass Incomplete", "Pass > 10 yds"]
    table["Run/Pass Scenarios"] = np.select(rpConditions, rpChoices)
    rpiChoices = table.to_csv("rpiChoices.csv")
    
    return table
scenarios()

C:\Users\User\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas

,,playNum,ydLine,DOWN,DIST,DD,GN/LS,PLAY TYPE,OFF FORM,OFF PLAY,PREVIOUS PLAY,prevPlay,currentPlay,Down/Dist Scenarios,Game Changing Scenarios,Run/Pass Scenarios
PLAY #,DN,,,,,,,,,,,,,,,
2.0,1.0,2.0,-29.0,1.0,10.0,1.010.0,4.0,Run,TWINS OPEN HARD,POWER,NaN,NaN,POWER,0,0,Run for 1st Down
3.0,2.0,3.0,-33.0,2.0,7.0,2.07.0,0.0,Pass,TWINS OPEN SOFT,PA POWER,POWER,POWER,PA POWER,0,0,Run < 5 yds
4.0,3.0,4.0,-33.0,3.0,7.0,3.07.0,-10.0,Pass,QUEENS OPEN,RB SCREEN WK,PA POWER,PA POWER,RB SCREEN WK,0,0,Run < 0 yds
5.0,3.0,5.0,-23.0,3.0,17.0,3.017.0,3.0,Run,QUEENS OPEN,QB POWER,RB SCREEN WK,RB SCREEN WK,QB POWER,0,0,Run < 5 yds
7.0,1.0,7.0,38.0,1.0,10.0,1.010.0,0.0,Run,SPREAD BONE T OVER,POWER,QB POWER,QB POWER,POWER,0,0,Run < 5 yds
8.0,2.0,8.0,38.0,2.0,10.0,2.010.0,2.0,Run,PRO WING,BUCK SWEEP,POWER,POWER,BUCK SWEEP,0,0,Run < 5 yds
9.0,3.0,9.0,36.0,3.0,8.0,3.08.0,5.0,Pass,QUADS,3ST,BUCK SWEEP,BUCK SWEEP,3ST,0,0,Pass Complete
10.0,4.0,10.0,31.0,4.0,2.0,4.02.0,3.0,Run,QUEENS OPEN,QB POWER,3ST,3ST,QB POWER,0,0,Run < 5 yds
11.0,1.0,11.0,25.0,1.0,10.0,1.010.0,1.0,Run,QUEENS OPEN,INF TRAP,QB POWER,QB POWER,INF TRAP,4th and Short Conversion,0,Run for 1st Down


In [18]:
def formation(*games):
    #Combine all xlsx files into 1 Dataframe
    all_data = pd.DataFrame()
    for f in glob.glob(r"C:\Users\User\Desktop\Personal_Projects\Gameplay_Analysis\*.xlsx"):
        df = pd.read_excel(f)
        all_data = all_data.append(df, ignore_index=True)
    all_games = all_data[["PLAY #","YARD LN","DN", "DIST", "OFF FORM",
                          "OFF PLAY", "PLAY TYPE", "GN/LS"]]
    form = pd.crosstab(all_games["OFF FORM"], 
                 all_games["OFF PLAY"])
    form.dropna()
    form
        
    return form
formation()

C:\Users\User\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


OFF PLAY,3 ST,3ST,5 ST,5ST,BUBBLE,BUBBLE SCREEN,BUCK SWEEP,BUCK SWEEP BOOT,CTR,CTR RD,...,SPRINT DRAW,SPRINT THROWBACK,T WRAP,TRAP,TRAP OPTION,TRIPLE OPTION,VEGAS,WING CTR,X TUNNEL,ZONE READ
OFF FORM,,,,,,,,,,,,,,,,,,,,,
3X2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3X2 BONE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3X2 WING,0,3,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACES,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DBL WING SPLIT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
EMPTY 3 X 2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
JOKER,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,3,0,0,0
KINGS,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
KINGS OPEN,2,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,2


In [42]:
 def fieldZones(*games):
    #Combine all xlsx files into 1 Dataframe
    all_data = pd.DataFrame()
    for f in glob.glob(r"C:\Users\User\Desktop\Personal_Projects\Gameplay_Analysis\*.xlsx"):
        df = pd.read_excel(f)
        all_data = all_data.append(df, ignore_index=True)
    all_games = all_data[["PLAY #","YARD LN","DN", "DIST", "OFF FORM",
                          "OFF PLAY", "PLAY TYPE", "GN/LS"]]
    
    #Sort columns and fill nan values
    table = all_games.dropna(subset=["OFF FORM"])
    table["DD"] = table[["DN", "DIST"]].astype(str).sum(axis=1)
    table["playNum"] = table["PLAY #"]
    table["DOWN"] = table["DN"]
#     table["ydLine"] = table["YARD LN"]
    table = table[["PLAY #","DN","playNum","YARD LN","DOWN",
                   "DIST","DD","GN/LS","PLAY TYPE", "OFF FORM", 
                   "OFF PLAY"]].set_index(["PLAY #", "DN"])
    table.fillna(value = table["GN/LS"].median(), inplace=True)

    #Field Zones
    fzConditions = [
    ((table["YARD LN"] >= -5.0) & (table["YARD LN"] <= 0.0)), 
    ((table["YARD LN"] <= -6.0) & (table["YARD LN"] >= -20.0)), 
    ((table["YARD LN"] <= -21.0) & (table["YARD LN"] >= -49.0)), 
    ((table["YARD LN"] <= 50.0) & (table["YARD LN"] >= 30.0)), 
    ((table["YARD LN"] <= 29.0) & (table["YARD LN"] >= 16.0)),
    ((table["YARD LN"] <= 15.0) & (table["YARD LN"] >= 6.0)),
    ((table["YARD LN"] <= 5.0) & (table["YARD LN"] >= 1.0))
    ]
    fzChoices = ["PI", "BU", "CO", "GI",
                  "LRZ", "HRZ", "GL"]
    table["FZ"] = np.select(fzConditions, fzChoices)
#     rpiChoices = table.to_csv("rpiChoices.csv")
    
    return table
fieldZones()

In [49]:
 def fieldZones(*games):
    #Combine all xlsx files into 1 Dataframe
    all_data = pd.DataFrame()
    for f in glob.glob(r"C:\Users\User\Desktop\Personal_Projects\Gameplay_Analysis\*.xlsx"):
        df = pd.read_excel(f)
        all_data = all_data.append(df, ignore_index=True)
    all_games = all_data[["PLAY #","YARD LN","DN", "DIST", "OFF FORM",
                          "OFF PLAY", "PLAY TYPE", "GN/LS"]]
    
    #Sort columns and fill nan values
    table = all_games.dropna(subset=["OFF FORM"])
    table["DD"] = table[["DN", "DIST"]].astype(str).sum(axis=1)
    table["playNum"] = table["PLAY #"]
    table["DOWN"] = table["DN"]
#     table["ydLine"] = table["YARD LN"]
    table = table[["PLAY #","DN","playNum","YARD LN","DOWN",
                   "DIST","DD","GN/LS","PLAY TYPE", "OFF FORM", 
                   "OFF PLAY"]].set_index(["PLAY #", "DN"])
    table.fillna(value = table["GN/LS"].median(), inplace=True)

    #Field Zones
    fzConditions = [
    ((table["YARD LN"] >= -5.0) & (table["YARD LN"] <= 0.0)), 
    ((table["YARD LN"] <= -6.0) & (table["YARD LN"] >= -20.0)), 
    ((table["YARD LN"] <= -21.0) & (table["YARD LN"] >= -49.0)), 
    ((table["YARD LN"] <= 50.0) & (table["YARD LN"] >= 30.0)), 
    ((table["YARD LN"] <= 29.0) & (table["YARD LN"] >= 16.0)),
    ((table["YARD LN"] <= 15.0) & (table["YARD LN"] >= 6.0)),
    ((table["YARD LN"] <= 5.0) & (table["YARD LN"] >= 1.0))
    ]
    fzChoices = ["PI", "BU", "CO", "GI",
                  "LRZ", "HRZ", "GL"]
    table["FZ"] = np.select(fzConditions, fzChoices)
#     rpiChoices = table.to_csv("rpiChoices.csv")
    
    fz = pd.crosstab([table["FZ"], table["OFF FORM"]], 
                 table["OFF PLAY"])
    fz.dropna()
    fz
        
    return fz
    
fieldZones()

C:\Users\User\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable

OFF PLAY                     3.0  3 ST  3ST  5 ST  5ST  BUBBLE  BUBBLE SCREEN  \
FZ  OFF FORM                                                                    
BU  ACES                       0     0    0     0    0       0              0   
    JOKER                      0     0    0     0    0       0              0   
    KINGS OPEN                 0     0    0     0    0       0              0   
    PRO SOFTER                 0     0    0     0    0       0              0   
    QUADS                      1     2    1     0    4       0              0   
    QUADS BONE                 0     0    0     0    0       0              0   
    QUEENS OPEN                0     0    1     1    0       0              0   
    SPREAD                     0     0    0     1    0       0              0   
    SPREAD BONE T OVER         0     0    0     0    0       0              0   
    TWINS OPEN HARD            0     0    0     0    1       0              0   
    TWINS OPEN HARDER          0     0    0     0    0       0              0   
    TWINS OPEN SOFTER          0     0    0     0    0       0              0   
    TWINS TITE HARDER          0     0    0     0    0       0              0   
    TWINS TITE SOFTER          0     0    0     0    0       0              0   
    VICTORY                    0     0    0     0    0       0              0   
CO  3X2 BONE                   0     0    0     0    0       0              0   
    ACES                       0     0    0     0    0       0              0   
    DBL WING SPLIT             0     0    0     0    0       0              0   
    EMPTY 3 X 2                0     0    0     0    0       0              0   
    JOKER                      0     0    0     0    0       0              0   
    KINGS                      0     0    0     0    0       0              0   
    KINGS OPEN                 0     2    0     0    0       0              0   
    KINGS WING UNB             0     0    0     0    0       0              0   
    MICKEY MOUSE               0     0    0     0    0       0              0   
    PRO HARDER                 0     0    0     0    0       0              0   
    PRO SOFTER                 0     0    0     0    0       0              0   
    PRO WING                   0     0    0     0    0       0              0   
    QUADS                      0     0    2     0    9       0              0   
    QUADS BONE                 0     0    0     0    0       0              0   
    QUEENS OPEN                0     0    2    10    1       1              0   
...                          ...   ...  ...   ...  ...     ...            ...   
HRZ QUADS BONE                 0     0    0     0    0       0              0   
    QUEENS OPEN                0     1    0     2    0       0              0   
    QUEENS OPEN BUNCH          0     0    0     0    0       0              0   
    QUEENS OPEN DOUBLE WING    0     0    0     0    0       0              0   
    QUEENS TITE                0     0    0     0    0       0              0   
    SPREAD                     0     0    0     0    0       0              0   
    SPREAD BONE T OVER         0     0    0     0    0       0              0   
    TWINS OPEN HARD            0     0    0     0    0       0              0   
    TWINS OPEN SOFTER          0     0    0     0    0       0              0   
    TWINS TITE HARDER          0     0    0     0    0       0              0   
    TWINS TITE SOFTER          0     0    0     0    0       0              0   
LRZ 3X2                        0     0    1     0    0       0              0   
    3X2 WING                   0     0    2     0    1       0              0   
    JOKER                      0     0    0     0    0       0              0   
    KINGS OPEN                 0     0    0     0    0       0              0   
    QUADS                      0     0    1     1    3       0              0   
    QUEENS OPEN                

In [51]:
def playCalls(*games):
    #Combine all xlsx files into 1 Dataframe
    all_data = pd.DataFrame()
    for f in glob.glob(r"C:\Users\User\Desktop\Personal_Projects\Gameplay_Analysis\*.xlsx"):
        df = pd.read_excel(f)
        all_data = all_data.append(df, ignore_index=True)
    all_games = all_data[["OFF FORM", "OFF PLAY", "PLAY TYPE", "BACK SET"]]
    plays = pd.crosstab(all_games["OFF PLAY"], 
                 [all_games["BACK SET"],
                  all_games["PLAY TYPE"],
                  all_games["OFF FORM"]])
    plays.dropna()
    plays
    plays.to_csv("PlayCalls_BackSet.csv")
    return plays
playCalls()

C:\Users\User\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


BACK SET                     FAR                                      PISTOL  \
PLAY TYPE                   Pass      Run                               Pass   
OFF FORM         UNBALANCED SLOT PRO WING UNBALANCED SLOT SPREAD BONE T OVER   
OFF PLAY                                                                       
3ST                            0        0               0                  0   
5ST                            0        0               0                  0   
BUCK SWEEP                     0        2               0                  0   
BUCK SWEEP BOOT                1        0               0                  0   
CTR                            0        0               0                  0   
CTR RD                         0        0               0                  0   
FLEA FLICKER                   0        0               0                  1   
G POWER                        0        0               0                  0   
INF TRAP                       0        0               0                  0   
JET RD                         0        0               0                  0   
JET SWEEP                      0        0               0                  0   
NO PLAY                        0        0               0                  0   
OZ                             0        0               0                  0   
PA POWER                       0        0               0                  0   
POWER                          0        0               0                  0   
POWER 3                        0        0               0                  0   
POWER BOOT                     0        0               0                  0   
POWER RD                       0        0               0                  0   
PULL BOOT                      0        0               0                  0   
QB BUCK SWEEP                  0        0               0                  0   
QB COUNTER                     0        0               0                  0   
QB DBL POWER                   0        0               0                  0   
QB DRAW                        0        0               0                  0   
QB LONG TRAP                   0        0               0                  0   
QB POWER                       0        0               0                  0   
QB POWER RPO                   0        0               0                  0   
QB T WRAP                      0        0               0                  0   
QB THROWBACK                   0        0               0                  0   
RB SCREEN STR                  0        0               0                  0   
RB SCREEN WK                   0        0               0                  0   
SPRINT                         0        0               0                  0   
SPRINT DRAW                    0        0               0                  0   
SPRINT THROWBACK               0        0               0                  0   
T WRAP                         0        0               0                  0   
TRAP                           0        0               1                  0   
TRAP OPTION                    0        0               0                  0   
VEGAS                          0        0               0                  0   
WING CTR                       0        0               0                  0   

BACK SET                                                            STRONG  \
PLAY TYPE                                       Run                   Pass   
OFF FORM         TWINS OPEN HARD SPREAD BONE T OVER TWINS OPEN SOFT  JOKER   
OFF PLAY                                                                     
3ST                            0                  0               0      0   
5ST                            2                  0               0      0   
BUCK SWEEP                     0                  0               0      0   
BUCK SWEEP BOOT                0                  0               0      0   
CTR            